In [1]:
import pandas as pd
# {"headers": {"Employee": ["id", "name", "salary", "departmentId"], "Department": ["id", "name"]}, "rows": 
#  {"Employee": [[1, "Joe", 85000, 1], [2, "Henry", 80000, 2], [3, "Sam", 60000, 2], [4, "Max", 90000, 1], [5, "Janet", 69000, 1], [6, "Randy", 85000, 1], [7, "Will", 70000, 1]],
#   "Department": [[1, "IT"], [2, "Sales"]]}}
pd.DataFrame([[1, "Joe", 85000, 1], [2, "Henry", 80000, 2], [3, "Sam", 60000, 2], [4, "Max", 90000, 1], [5, "Janet", 69000, 1], [6, "Randy", 85000, 1], [7, "Will", 70000, 1]], columns=["id", "name", "salary", "departmentId"]).to_csv("./Employee.txt", index=None)
pd.DataFrame([[1, "IT"], [2, "Sales"]], columns = ["id", "name"]).to_csv("./Department.txt", index=None)

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("department top three salaries").config("pyspark.sql.shuffle.partition", "4").getOrCreate()

In [4]:
e = spark.read.option("header", True).csv("./Employee.txt")
d = spark.read.option("header", True).csv("./Department.txt")
e.show()
d.show()

+---+-----+------+------------+
| id| name|salary|departmentId|
+---+-----+------+------------+
|  1|  Joe| 85000|           1|
|  2|Henry| 80000|           2|
|  3|  Sam| 60000|           2|
|  4|  Max| 90000|           1|
|  5|Janet| 69000|           1|
|  6|Randy| 85000|           1|
|  7| Will| 70000|           1|
+---+-----+------+------------+

+---+-----+
| id| name|
+---+-----+
|  1|   IT|
|  2|Sales|
+---+-----+



A company's executives are interested in seeing who earns the most money in each of the company's departments. A high earner in a department is an employee who has a salary in the top three unique salaries for that department.

Write an SQL query to find the employees who are high earners in each of the departments.

Return the result table in any order.

The query result format is in the following example.

In [16]:
from pyspark.sql.window import Window
from pyspark.sql.functions import dense_rank, col

window_spec = Window.partitionBy("departmentId").orderBy(col("salary").desc())
e.select("*", dense_rank().over(window_spec).alias("rank")).where("rank <= 3").join(d, e.departmentId==d.id).select(d.name.alias("Department"), e.name.alias("Employee"), e.salary).show()

+----------+--------+------+
|Department|Employee|salary|
+----------+--------+------+
|        IT|     Max| 90000|
|        IT|     Joe| 85000|
|        IT|   Randy| 85000|
|        IT|    Will| 70000|
|     Sales|   Henry| 80000|
|     Sales|     Sam| 60000|
+----------+--------+------+

